
<img src="6.jpeg" width="500">
<img src="5.jpeg" width="500">


# <span style="color: springgreen;">🌾Crop Production </span>

## <span style="color: springgreen;">✒️Objective </span>

The primary objective of analyzing this dataset is to understand the factors influencing crop production and to develop predictive models that can accurately forecast crop yields. This analysis aims to provide valuable insights to farmers, agricultural policymakers, and researchers to optimize agricultural practices and improve productivity.

## <span style="color: springgreen;">✒️Data Overview </span>

**The Dataset contains several key features of various crops produced in India. Below is a description for each features.**
- **Crop_Year**: The year in which the crop was harvested.
- **Season**: The season during which the crop was grown (e.g., Kharif, Rabi, Whole Year).
- **Crop**: The type of crop grown.
- **Area**: The area of land (in hectares) used for cultivation.
- **Production**: The total production of the crop (in tonnes).


## <span style="color: springgreen;">✒️Import Required Libraries </span>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

In [2]:
#reading the dataset
df=pd.read_csv('Crop Production data.csv')

In [3]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [4]:
df.tail()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
246086,West Bengal,PURULIA,2014,Summer,Rice,306.0,801.0
246087,West Bengal,PURULIA,2014,Summer,Sesamum,627.0,463.0
246088,West Bengal,PURULIA,2014,Whole Year,Sugarcane,324.0,16250.0
246089,West Bengal,PURULIA,2014,Winter,Rice,279151.0,597899.0
246090,West Bengal,PURULIA,2014,Winter,Sesamum,175.0,88.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246091 entries, 0 to 246090
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   State_Name     246091 non-null  object 
 1   District_Name  246091 non-null  object 
 2   Crop_Year      246091 non-null  int64  
 3   Season         246091 non-null  object 
 4   Crop           246091 non-null  object 
 5   Area           246091 non-null  float64
 6   Production     242361 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 13.1+ MB


> **Note**-
The dataset contains 7 features includes 4 Nominal Categorical Variables,a date column and 2 Numerical Columns. It also have several Null values.

In [6]:
df.describe()

,Crop_Year,Area,Production
count,246091.000000,2.460910e+05,2.423610e+05
mean,2005.643018,1.200282e+04,5.825034e+05
std,4.952164,5.052340e+04,1.706581e+07
min,1997.000000,4.000000e-02,0.000000e+00
25%,2002.000000,8.000000e+01,8.800000e+01
50%,2006.000000,5.820000e+02,7.290000e+02
75%,2010.000000,4.392000e+03,7.023000e+03
max,2015.000000,8.580100e+06,1.250800e+09


In [7]:
df.duplicated().sum()

0

In [24]:
#Check for the null value presence in the dataset
round(df.isna().sum()/df.shape[0],3)*100

State_Name       0.0
District_Name    0.0
Crop_Year        0.0
Season           0.0
Crop             0.0
Area             0.0
Production       1.5
dtype: float64

> **Note**- 
There are 1.5% null values presents in the dataset, we can delete them as they have minimal impact on the Dataset.

## <span style="color: springgreen;">✒️Exploratory Data Analysis </span>

In [9]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [10]:
#Add the categorical and numerical features in Separate lists
cats=[col for col in df.columns if df[col].dtype=="O"]
nums=['Area','Production']

In [26]:
#check for the unique values in Season
df['Season'].unique()

array(['Kharif     ', 'Whole Year ', 'Autumn     ', 'Rabi       ',
       'Summer     ', 'Winter     '], dtype=object)

In [27]:
#Remove the extra spaces from the Values in Season
df['Season']=df['Season'].apply(lambda x : x.strip())
df['Season'].unique()

array(['Kharif', 'Whole Year', 'Autumn', 'Rabi', 'Summer', 'Winter'],
      dtype=object)

In [28]:
#Check for the value counts in Season Features
df['Season'].value_counts()

Season
Kharif        95951
Rabi          66987
Whole Year    57305
Summer        14841
Winter         6058
Autumn         4949
Name: count, dtype: int64